In [1]:
import os
from os import listdir
from os.path import isfile, join
import re
import sys


#SPAM PATH HAM PATH NORM DICT PATH SPAM DICT PATH GLOBAL VARIABLES 
SPAM_PATH = os.path.join("HamandSpam", "Spam", "spam") 

HAM_PATH = os.path.join("HamandSpam", "Ham", "easy_ham") 

NORMAL_DICTIONARY_PATH = os.path.join("Dictionaries", "dictionary.txt") 

SPAM_DICTIONARY_PATH = os.path.join("Dictionaries", "spamdictionary.txt") 

#HYPERPARAMETERS 
REMOVE_PUNCTUATION = True
REMOVE_SYMBOLS = True
# if False count how many times a word shows up per word
# if True count words only once, whether they appear or don't
COUNT_SINGLE_INSTANCE = False
SET_ALL_WORDS_UPPER_CASE = True
COUNT_INVALID_LONE_LETTERS = True
CHECK_IF_EMAIL_CONTAINS_MARKUP = True
COUNT_UPPER_CASE_WORDS_BEFORE_PROCESSING = True
COUNT_ALL_SYMBOL_WORDS_LENGTH_THRESHOLD = [True, 2]
#some emails have rather long words, especially spam
#this leads to a large dataset when looking at unique words
#setting shortest and longest word limits help reduce dataset size
REMOVE_WORDS_LENGTH_THRESHOLD = [True, 4, 8]
CHECK_MARKUP_TEXT_PRESENCE = True

#HELPER FUNCTIONS 
def removeAllSymbolsFromAString(word):
    for letter in word:
        if(isSymbol(letter)):
            word = word.replace(letter, "")

    return word


def removeAllPunctFromAString(word):
    for letter in word:
        if(isPunctuation(letter)):
            word = word.replace(letter, "")

    return word
    

def isPunctuation(letter):
    punctuation_text = re.search(r"([/./?\,\'\-\—\!\:\;\[\]\(\)\/\\\"])", letter)
    if(punctuation_text is None):
        return False
    else:
        return True
    

# checker function for making sure invalid lone letter count only
# increments for actual letters
def isLetter(letter):
    letter_text = re.search(r"([A-Za-z])", letter)
    if(letter_text is None):
        return False
    else:
        return True

# capital letter checker, same as above function but don't capitalize
# input prior to checking
def isCapitalLetter(letter):
    letter_text = re.search(r"([A-Z])", letter)
    if(letter_text is None):
        return False
    else:
        return True


def isUpperCaseWord(word):
    # remove symbols and punctuation since only letters can be capital
    word = removeAllPunctFromAString(word)
    word = removeAllSymbolsFromAString(word)
    # if the word was just symbols and punctuation
    # return False if the word was all symbols and punctuation
    if(len(word) == 0):
        return False
    sawACapitalLetter = False
    for letter in word:
        if(isCapitalLetter(letter) == False):
            return False

    return True


def isSymbol(letter):
    symbol_text = re.search(r"([\?\/\\\|\~\`\<\>\,\.\:\;\\\'\{\}\-\—\=\+\!\@\#\\\$\%\^\&\*])", letter)
    if(symbol_text is None):
        return False
    else:
        return True
def isAllSymbolWord(word):
    for letter in word:
        if(letter == " " or letter == ""):
            return False
        if(isSymbol(letter) != True):
            return False
    return True
def isMarkupWord(word):
    markup_text = re.search(r"\<([A-Za-z0-9_]+)\>", word)
    if(markup_text is None):
        return False
    else:
        return True
#keys used in the containers for data that shouldn't be overwritten if an email contains them
def isSpecialWord(word):
    if(word == "upperCnt" or word == "symbolCnt" or word == "loneCnt" or word = "markupPrsnt" or word = "IsSpam"):
        print("Special Word was present")
        sys.Exit()
        return True
    else:
        return False


def GetDataApplyHyperparameters(spam_path = SPAM_PATH, ham_path = HAM_PATH, norm_dict_path = NORMAL_DICTIONARY_PATH,
                               spam_dict_path = SPAM_DICTIONARY_PATH, remove_punctuation = REMOVE_PUNCTUATION,
                        remove_symbols = REMOVE_SYMBOLS, count_single_instance = COUNT_SINGLE_INSTANCE,
                        set_all_words_upper_case = SET_ALL_WORDS_UPPER_CASE, 
                         count_invalid_lone_letters = COUNT_INVALID_LONE_LETTERS, 
                        count_upper_case_words_before_processing = COUNT_UPPER_CASE_WORDS_BEFORE_PROCESSING,
                        count_all_symbol_words_length_threshold = COUNT_ALL_SYMBOL_WORDS_LENGTH_THRESHOLD,
                        remove_words_length_threshold = REMOVE_WORDS_LENGTH_THRESHOLD, 
                                check_markup_text_presence = CHECK_MARKUP_TEXT_PRESENCE):
    
    
    onlyfiles = [f for f in listdir(spam_path) if isfile(os.path.join(spam_path, f))]

    spam_word_count_per_email_data = {}

    email_counter = 1


    for filename in onlyfiles:

        dictionaryName = "Email" + str(email_counter)

        spam_word_count_per_email_data[dictionaryName] = {}

        encodings = ['utf-8', 'windows-1250', 'windows-1252', 'ascii']
        for en in encodings:
            try:

                file1 = open(os.path.join(spam_path, filename), 'r', encoding=en)

                alllines = file1.read()
                alllinessplit = alllines.split()

                invalidLoneLetterCount = 0
                preProccessUpperCaseCount = 0
                allSymbolWordsCount = 0

                sawMarkupWord = False

                for word in alllinessplit:
                    if(isinstance(word, str)):

                        if(isSpecialWord(word)):
                            continue
                        
                        if(word == " " or word == ""):
                            continue

                        #markup check should be done before all processing
                        if(check_markup_text_presence):
                            if(isMarkupWord(word)):
                                sawMarkupWord = True
                                continue
                                
                            
                        # counts that need to be done pre proccessing
                        if(count_upper_case_words_before_processing):
                            if(isUpperCaseWord(word)):
                                preProccessUpperCaseCount = preProccessUpperCaseCount + 1
                        if(count_all_symbol_words_length_threshold[0]):
                            if(len(word) >= count_all_symbol_words_length_threshold[1]):
                                if(isAllSymbolWord(word)):
                                    allSymbolWordsCount = allSymbolWordsCount + 1
                                    continue
                                    

                        # hyper parameter filters
                        wordAfterHyperParameters = word
                        if remove_punctuation:
                            wordAfterHyperParameters = removeAllPunctFromAString(
                                wordAfterHyperParameters)
                        if remove_symbols:
                            wordAfterHyperParameters = removeAllSymbolsFromAString(
                                wordAfterHyperParameters)
                        if set_all_words_upper_case:
                            wordAfterHyperParameters = wordAfterHyperParameters.upper()

                        # count to do after processing
                        if len(wordAfterHyperParameters) == 1 and isLetter(
                                wordAfterHyperParameters):

                            if(count_invalid_lone_letters):
                                if(wordAfterHyperParameters != "i" and wordAfterHyperParameters != "I" and wordAfterHyperParameters != "a" and wordAfterHyperParameters != "A"):
                                    invalidLoneLetterCount = invalidLoneLetterCount + 1

                        # count occurrences only for the hyperparameter being set
                        if wordAfterHyperParameters in spam_word_count_per_email_data[
                                dictionaryName] and count_single_instance == False:
                            spam_word_count_per_email_data[dictionaryName][wordAfterHyperParameters] = spam_word_count_per_email_data[
                                dictionaryName][wordAfterHyperParameters] + 1
                        elif (len(wordAfterHyperParameters) > remove_words_length_threshold[1] and 
                             len(wordAfterHyperParameters) < remove_words_length_threshold[2]):
                            spam_word_count_per_email_data[dictionaryName][wordAfterHyperParameters] = 1
                email_counter = email_counter + 1
                #if hyperparameters were enabled add their respective values to the container
                if(count_invalid_lone_letters):
                    spam_word_count_per_email_data[dictionaryName]["loneCnt"] = invalidLoneLetterCount
                if(count_upper_case_words_before_processing):
                    spam_word_count_per_email_data[dictionaryName]["upperCnt"] = preProccessUpperCaseCount
                if(count_all_symbol_words_length_threshold):
                    spam_word_count_per_email_data[dictionaryName]["symbolCnt"] = allSymbolWordsCount
                if(check_markup_text_presence):
                    if(sawMarkupWord):
                        spam_word_count_per_email_data[dictionaryName]["markupPrsnt"] = 1
                    else:
                        spam_word_count_per_email_data[dictionaryName]["markupPrsnt"] = 0
                file1.close()
            except UnicodeDecodeError:
                print('got unicode error with %s , trying different encoding' % en)
            else:
                break

    onlyfiles = [f for f in listdir(ham_path) if isfile(os.path.join(ham_path, f))]


    ham_word_count_per_email_data = {}

    email_counter = 1

    for filename in onlyfiles:

        dictionaryName = "Email" + str(email_counter)

        ham_word_count_per_email_data[dictionaryName] = {}

        encodings = ['utf-8', 'windows-1250', 'windows-1252', 'ascii']
        for en in encodings:
            try:

                file1 = open(os.path.join(ham_path, filename), 'r', encoding=en)

                alllines = file1.read()
                alllinessplit = alllines.split()

                invalidLoneLetterCount = 0
                preProccessUpperCaseCount = 0
                allSymbolWordsCount = 0
                
                sawMarkupWord = False
                
                for word in alllinessplit:
                    if(isinstance(word, str)):
                        
                        if(isSpecialWord(word)):
                            continue

                        if(word == " " or word == ""):
                            continue
                            
                        #markup check should be done before all processing
                        if(check_markup_text_presence):
                            if(isMarkupWord(word)):
                                sawMarkupWord = True
                                continue
                                    
                        # counts that need to be done pre proccessing
                        if(count_upper_case_words_before_processing):
                            if(isUpperCaseWord(word)):
                                preProccessUpperCaseCount = preProccessUpperCaseCount + 1
                        if(count_all_symbol_words_length_threshold[0]):
                            if(len(word) > count_all_symbol_words_length_threshold[1]):
                                if(isAllSymbolWord(word)):
                                    allSymbolWordsCount = allSymbolWordsCount + 1
                                    continue
                                    

                        # hyper parameter filters
                        wordAfterHyperParameters = word
                        if remove_punctuation:
                            wordAfterHyperParameters = removeAllPunctFromAString(
                                wordAfterHyperParameters)
                        if remove_symbols:
                            wordAfterHyperParameters = removeAllSymbolsFromAString(
                                wordAfterHyperParameters)
                        if set_all_words_upper_case:
                            wordAfterHyperParameters = wordAfterHyperParameters.upper()

                        # count to do after processing
                        if len(wordAfterHyperParameters) == 1 and isLetter(
                                wordAfterHyperParameters):

                            if(count_invalid_lone_letters):
                                if(wordAfterHyperParameters != "i" and wordAfterHyperParameters != "I" and wordAfterHyperParameters != "a" and wordAfterHyperParameters != "A"):
                                    invalidLoneLetterCount = invalidLoneLetterCount + 1

                        # count occurrences only for the hyperparameter being set
                        if wordAfterHyperParameters in ham_word_count_per_email_data[
                                dictionaryName] and count_single_instance == False:

                            ham_word_count_per_email_data[dictionaryName][wordAfterHyperParameters] = ham_word_count_per_email_data[
                                dictionaryName][wordAfterHyperParameters] + 1
                        elif (len(wordAfterHyperParameters) > remove_words_length_threshold[1] and 
                             len(wordAfterHyperParameters) < remove_words_length_threshold[2]):
                            ham_word_count_per_email_data[dictionaryName][wordAfterHyperParameters] = 1
                email_counter = email_counter + 1
                if(count_invalid_lone_letters):
                    ham_word_count_per_email_data[dictionaryName]["loneCnt"] = invalidLoneLetterCount
                if(count_upper_case_words_before_processing):
                    ham_word_count_per_email_data[dictionaryName]["upperCnt"] = preProccessUpperCaseCount
                if(count_all_symbol_words_length_threshold[0]):
                    ham_word_count_per_email_data[dictionaryName]["symbolCnt"] = allSymbolWordsCount
                if(check_markup_text_presence):
                    if(sawMarkupWord):
                        ham_word_count_per_email_data[dictionaryName]["markupPrsnt"] = 1
                    else:
                        ham_word_count_per_email_data[dictionaryName]["markupPrsnt"] = 0
                file1.close()
            except UnicodeDecodeError:
                print('got unicode error with %s , trying different encoding' % en)
            else:
                break


    
    
    print("DONE READING ALL EMAIL FILES",  "Email counts Spam: ",
        len(spam_word_count_per_email_data), " Ham: ",
        len(ham_word_count_per_email_data))


    dictionary = open(norm_dict_path, "r", encoding='utf-8')

    all_norm_dictionary_words = []

    alllines = dictionary.read()
    alllinessplit = alllines.split()

    for word in alllinessplit:
        # upper case for the purpose of comparison later
        all_norm_dictionary_words.append(word.upper())

    dictionary.close()


    

    spam_dictionary = open(spam_dict_path, "r", encoding='utf-8')

    all_spam_dictionary_words = []

    alllinesspam = spam_dictionary.read()
    alllinessplitspam = alllinesspam.split()

    for word in alllinessplitspam:
        # upper case for the purpose of comparison later
        all_spam_dictionary_words.append(word.upper())

    spam_dictionary.close()

    print("DONE READING ALL DICTIONARY FILES", "Length Normal Dict: ", len(all_norm_dictionary_words),
         " Length Spam Dict: ", len(all_spam_dictionary_words))

    return spam_word_count_per_email_data, ham_word_count_per_email_data, all_norm_dictionary_words, all_spam_dictionary_words


spam_word_count_container, ham_word_count_container, norm_dict, spam_dict = GetDataApplyHyperparameters()




got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , 

got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , trying different encoding
got unicode error with utf-8 , 

In [23]:


# allpossiblewords = []

totalWordCount = 0

for email in spam_word_count_container:
    for word in spam_word_count_container[email]:
        if(isMarkupWord(word) or isAllSymbolWord(word) or isPunctuation(word)):
            print(word)
        totalWordCount = totalWordCount + 1

for email in ham_word_count_container:
    for word in ham_word_count_container[email]:
        if(isMarkupWord(word) or isAllSymbolWord(word) or isPunctuation(word)):
            print(word)
        totalWordCount = totalWordCount + 1

        
# for email in spam_word_count_container:
#     if(spam_word_count_container[email]["markupPrsnt"] == 1):
#         print(spam_word_count_container[email])
# #         sys.exit()
#     for word in spam_word_count_container[email]:
#         if(allpossiblewords.count(word) == 0):
#             # if a threshold for how long a word is to be counted is specified
#             # check the word is valid length
#             if(REMOVE_WORDS_LENGTH_THRESHOLD[0]):
#                 if(len(word) > REMOVE_WORDS_LENGTH_THRESHOLD[1]):
#                     allpossiblewords.append(word)
#                    # print(len(allpossiblewords), "spam addition")
                
# sys.exit()
# for email in ham_word_count_container:

#     for word in ham_word_count_container[email]:
#         if(allpossiblewords.count(word) == 0):
#             # if a threshold for how long a word is to be counted is specified
#             # check the word is valid length
#             if(REMOVE_WORDS_LENGTH_THRESHOLD[0]):
#                 if(len(word) > REMOVE_WORDS_LENGTH_THRESHOLD[1]):
#                     allpossiblewords.append(word)
#                     print(len(allpossiblewords), "ham addition")
              


# print(len(allpossiblewords), "total words")




In [24]:
print(totalWordCount)

196181


In [36]:
import random
#to generate random indices when picking words from an email
def randomListOfXValuesInRange(high_lim, number_items_to_return):
    return_vals = []

    if(number_items_to_return <= 0):
        return return_vals
    
    while len(return_vals) < number_items_to_return:
        random_num = random.randrange(high_lim)
        if(return_vals.count(random_num) == 0):
            return_vals.append(random_num)
    return return_vals




def SamplePortionFromEveryEmail(container_spam, container_ham, portion_size):
    
    possible_words_for_sample = []
    
    for email in container_spam:
        total_words_in_email = len(container_spam[email])
        
        random_indices = randomListOfXValuesInRange(total_words_in_email, int(round(portion_size*total_words_in_email)))
        current_index = 0
        for word in container_spam[email]:
            if(random_indices.count(current_index) > 0 ):
                possible_words_for_sample.append(word)
            current_index = current_index + 1
            
    for email in container_ham:
        total_words_in_email = len(container_ham[email])
        
        random_indices = randomListOfXValuesInRange(total_words_in_email, int(round(portion_size*total_words_in_email)))
        current_index = 0
        for word in container_ham[email]:
            if(random_indices.count(current_index) > 0 ):
                possible_words_for_sample.append(word)
            current_index = current_index + 1
    
    return possible_words_for_sample


words_from_sample = SamplePortionFromEveryEmail(spam_word_count_container, ham_word_count_container, 0.1)

print(len(words_from_sample))



19635


In [37]:
import pandas as pd
import numpy as np

print(type(words_from_sample))

<class 'list'>


In [38]:
#Let 1 = spam and 0 = ham
all_train_rows = []

for email in spam_word_count_container:
    single_train_row = []
    
    #check if that email has the word, 
    #1 if it does, 0 if not
    for word in words_from_sample:
        if(word in spam_word_count_container[email]):
            single_train_row.append(1)
        else:
            single_train_row.append(0)
    #append 1 for spam for last column which is "IsSpam"
    single_train_row.append(1)
    all_train_rows.append(single_train_row)
    
for email in ham_word_count_container:
    single_train_row = []
    
    #check if that email has the word, 
    #1 if it does, 0 if not
    for word in words_from_sample:
        if(word in ham_word_count_container[email]):
            single_train_row.append(1)
        else:
            single_train_row.append(0)
    #append 0 for ham for last column which is "IsSpam"
    single_train_row.append(0)
    all_train_rows.append(single_train_row)
    
    
    
  
    
    
    
 
    
    
    



In [39]:
print(len(all_train_rows))
print(len(words_from_sample))

3047
19635


In [40]:
is_spam_col = "IsSpam"
words_from_sample.append(is_spam_col)
print(len(words_from_sample))

19636


In [41]:
Train_df = pd.DataFrame(all_train_rows, columns = words_from_sample)

In [42]:
Train_df["IsSpam"].value_counts()

0    2551
1     496
Name: IsSpam, dtype: int64

In [43]:
Train_df.head()

,ESMTP,TABLE,GOTHIC,SAVINGS,FAMILY,ERROR,POSTFIX,131944,MIMEOLE,LINUX,...,METHOD,123229,080410,123231,upperCnt,GROUND,upperCnt,COURSE,symbolCnt,IsSpam
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,1
1,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,1,0,1,0,1,1
2,1,0,0,0,0,0,1,1,1,1,...,0,0,0,0,1,0,1,0,1,1
3,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,1,0,1,1
4,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,1,0,1,0,1,1


In [44]:
X_train = Train_df.drop("IsSpam", axis = 1)

In [45]:
y_train = Train_df["IsSpam"]

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
cross_val_score(forest_clf, X_train, y_train, cv=3,
                                    scoring="accuracy")

array([0.84055118, 0.96850394, 0.96059113])